In [ ]:
# Celda 1 — Configuración e imports (Entropía espectral H)

from pathlib import Path
import numpy as np
import pandas as pd

# Carpeta con los .wav (una subcarpeta por especie)
IN_DIR = "../data/Tinamidae"

# Archivo de salida
OUT_CSV = "../traits/output_csv/H.csv"

# Audio
SR_TARGET = 44100   # Hz
EPS = 1e-12

# Frames 
WIN_LENGTH  = 1024
HOP_LENGTH  = WIN_LENGTH // 2  # 50% solape

# Entropía (antropy.spectral_entropy, método Welch)
USE_ANTROPY   = True   # si no está instalada, se usa fallback con SciPy
ENT_NORMALIZE = True   # entropía normalizada a [0,1] (Shannon / log(Nbins))

In [ ]:
# Celda 2 — Utilidades: I/O, z-score, frames y entropía H sobre PSD-Welch

import librosa
from scipy.signal import welch

def list_wavs(base_dir: str):
    base = Path(base_dir)
    return sorted([p for p in base.rglob("*.wav") if p.is_file()])

def species_from_path(p: Path, base_dir: str):
    """
    Asume estructura: base_dir/especie/archivo.wav
    Devuelve el nombre de la carpeta inmediatamente bajo base_dir.
    """
    base = Path(base_dir).resolve()
    rel  = p.resolve().relative_to(base)
    return rel.parts[0] if len(rel.parts) > 1 else "UNKNOWN"

def load_mono(fp: Path):
    """
    Carga audio mono a SR_TARGET.
    """
    y, sr = librosa.load(str(fp), sr=SR_TARGET, mono=True)
    return y.astype(np.float64, copy=False), SR_TARGET

def zscore_signal(y: np.ndarray, eps: float = EPS):
    """
    Estandariza la señal: (y - media) / sigma.
    """
    if y.size == 0:
        return y
    mu = float(np.mean(y))
    sd = float(np.std(y, ddof=0))
    return (y - mu) / (sd + eps)

def estimate_frames(n_samples: int, win: int = WIN_LENGTH, hop: int = HOP_LENGTH) -> int:
    """
    Estima el número de frames T dado win y hop.
    """
    if n_samples < win:
        return 1
    return 1 + (n_samples - win) // hop

# Intento de usar antropy; si no está, fallback SciPy
try:
    from antropy import spectral_entropy as _ant_spectral_entropy
    _HAS_ANTROPY = True
except Exception:
    _HAS_ANTROPY = False

def spectral_entropy_welch(y_hat: np.ndarray, sr: int, normalize: bool = True) -> float:
    """
    Entropía de Shannon sobre PSD-Welch normalizada (tratada como distribución de probabilidad).

    - Si antropy está disponible y USE_ANTROPY=True:
        usa antropy.spectral_entropy(y_hat, sf=sr, method="welch", normalize=normalize)
    - Si no, hace:
        1) Welch para estimar PSD
        2) Normaliza PSD para que sume 1
        3) H = -sum p * log(p)
        4) Si normalize=True, divide por log(N_bins)
    """
    if y_hat.size == 0:
        return np.nan

    if USE_ANTROPY and _HAS_ANTROPY:
        H = _ant_spectral_entropy(
            y_hat, sf=sr, method="welch", nperseg=2048, normalize=normalize
        )
        return float(H)

    # Fallback: SciPy Welch + Shannon
    f, Pxx = welch(
        y_hat, fs=sr, nperseg=2048, noverlap=1024,
        window="hann", scaling="density"
    )
    Pxx = np.maximum(Pxx, 0.0)
    psum = np.sum(Pxx) + EPS
    p = Pxx / psum                     # distribución de prob sobre las frecuencias
    H = -np.sum(p * np.log(p + EPS))   # Shannon
    if normalize:
        H = H / np.log(p.size)
    return float(H)


In [ ]:
# Celda 3 — Cálculo de H crudo (H_raw) por archivo

files = list_wavs(IN_DIR)
print(f" WAVs encontrados: {len(files)}")

rows = []

for fp in files:
    sp = species_from_path(fp, IN_DIR)
    try:
        y, sr    = load_mono(fp)                # 44.1 kHz mono
        y_hat    = zscore_signal(y)             # señal canónica
        H_raw    = spectral_entropy_welch(y_hat, sr, normalize=ENT_NORMALIZE)
        T_frames = int(estimate_frames(len(y_hat), WIN_LENGTH, HOP_LENGTH))

        rows.append({
            "species": sp,
            "relpath": str(fp),
            "sr": sr,
            "T_frames": T_frames,
            "H_raw": H_raw,
        })
    except Exception as e:
        print(f"[WARN] Error en {fp}: {e}")
        rows.append({
            "species": sp,
            "relpath": str(fp),
            "sr": np.nan,
            "T_frames": 0,
            "H_raw": np.nan,
            "error": f"{type(e).__name__}: {e}"
        })

df_H_files = pd.DataFrame(rows)
print("Archivos procesados:", df_H_files.shape[0])
df_H_files.head()


In [ ]:
# Celda 4 — Ajuste log(T) y residuales: H_hat = H_raw - (a + b*log(T))

df_fit = df_H_files.dropna(subset=["H_raw", "T_frames"]).copy()
df_fit = df_fit[df_fit["T_frames"] > 1].copy()

df_fit["log_T"] = np.log(df_fit["T_frames"].astype(float))

x = df_fit["log_T"].values
y = df_fit["H_raw"].values

# Ajuste lineal: H_raw ~ a + b*log(T_frames)
b, a = np.polyfit(x, y, 1)  # slope=b, intercept=a

print(f"Coeficientes de corrección (MCO) usando N = {len(x)} audios:")
print(f"  a (intercepto) = {a:.6f}")
print(f"  b (pendiente)  = {b:.6f}")
print("Referencia previa (dataset completo): a ≈ 0.496105, b ≈ -0.038873")

# Residuales: H_hat
df_fit["H_hat"] = df_fit["H_raw"] - (a + b * df_fit["log_T"])

print("\nEjemplo de primeras filas con H_raw y H_hat:")
df_fit[["species", "T_frames", "H_raw", "H_hat"]].head()


In [ ]:
# Celda 5 — Resumen por especie y export de H.csv

df_species = (
    df_fit
    .dropna(subset=["H_raw", "H_hat"])
    .groupby("species")
    .agg(
        n_muestras    = ("relpath", "count"),
        T_frames_mean = ("T_frames", "mean"),
        H_raw_mean    = ("H_raw", "mean"),
        H_raw_median  = ("H_raw", "median"),
        H_hat_mean    = ("H_hat", "mean"),
        H_hat_median  = ("H_hat", "median"),
    )
    .reset_index()
    .sort_values("n_muestras", ascending=False)
)

print("Especies con datos:", df_species.shape[0])
df_species.head(10)

df_species.to_csv(OUT_CSV, index=False)
print(f"[OK] Archivo guardado en: {OUT_CSV}")

print("\nModo entropía:",
      "Antropy (Welch)" if (USE_ANTROPY and _HAS_ANTROPY) else "SciPy Welch (fallback)")
